# Trouble controlling `Scatter.selected`

---
## Normal usage, works fine

Clicking a node adds it to the selected list

In [11]:
import bqplot as bq

In [12]:
scales = {
    'x': bq.LinearScale(min=0, max=10),
    'y': bq.LinearScale(min=0, max=10),
}

x_vals = [2,5,8]
y_vals = [2,8,2]

In [13]:
scatter = bq.Scatter(
    x=x_vals, y=y_vals,
    scales=scales,
    default_size=180,
    interactions={'click': 'select'},
    selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
    unselected_style={'opacity': 0.5})

In [14]:
# Selecting multiple with ctrl + click
bq.Figure(marks=[scatter])

A Jupyter Widget

In [15]:
scatter.selected

[]

---
## My weird usage

I was tyring to implement MVC pattern, similar to this, where clicking a node would add it to the model's list of selected nodes, and all selected nodes would be highlighted. Programatically changing `selected` property causes problems.

In [16]:
import bqplot as bq
import traitlets as t
import traittypes as tt
import ipywidgets as ipyw

In [17]:
class ScatterModel(t.HasTraits):
    sel_list = t.List(trait=t.Int(), default_value=[])
    
    def __init__(self, *args, **kwargs):
        super(ScatterModel, self).__init__(*args, **kwargs)

In [18]:
class ScatterView(ipyw.VBox):
    
    def __init__(self, *args, **kwargs):
        super(ScatterView, self).__init__(*args, **kwargs)
        
        # Set up Scatter
        self.model = ScatterModel()
        scales = {
            'x': bq.LinearScale(min=0, max=10),
            'y': bq.LinearScale(min=0, max=10),
        }
        x_vals = [2,5,8]
        y_vals = [2,8,2]
        self.scatter = bq.Scatter(
            x=x_vals, y=y_vals,
            scales=scales,
            default_size=180,
            interactions={'click': 'select'},
            selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
            unselected_style={'opacity': 0.5}
        )
        
        # Add it to view
        fig = bq.Figure(marks=[self.scatter])
        self.children = [fig]
        
        # Sync up model and view
        self.scatter.on_element_click(self.callback_click)
        t.dlink((self.model, 'sel_list'), (self.scatter, 'selected'))
        
    def callback_click(self, scatter, change):
        """When an element is clicked, add its index to model.sel_list"""
        
        idx = change['data']['index']
        
        tmp = self.model.sel_list[:]
        tmp +=[idx]
        self.model.sel_list = tmp

In [19]:
view = ScatterView()

In [20]:
# Selecting mutliple with normal click
view

A Jupyter Widget

In [23]:
view.model.sel_list

[1, 0]

In [25]:
view.scatter.selected

[0]

This is a very partial implementation, but you can see the basic issue:

> Clicking a node adds it to the model (`model.sel_list`), and that *appears* to sync up with `scatter.selected`, as all the clicked nodes are highlighted. However, the `scatter.selected` property does not actually change, creating some problems down the road.

---
## Solution

Simply removing the line that sets `interactions={'click': 'select'}`solved the issue, allowing me to control the selected nodes programatically, based on the model, without weird side effects.

In [13]:
class ScatterView2(ipyw.VBox):
    
    def __init__(self, *args, **kwargs):
        super(ScatterView2, self).__init__(*args, **kwargs)
        
        self.model = ScatterModel()
        
        scales = {
            'x': bq.LinearScale(min=0, max=10),
            'y': bq.LinearScale(min=0, max=10),
        }

        x_vals = [2,5,8]
        y_vals = [2,8,2]
        
        self.scatter = bq.Scatter(
            x=x_vals, y=y_vals,
            scales=scales,
            default_size=180,
#             interactions={'click': 'select'},
            selected_style={'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'},
            unselected_style={'opacity': 0.5}
        )
        
        fig = bq.Figure(marks=[self.scatter])
        self.children = [fig]
        
        # Sync up model and view
        self.scatter.on_element_click(self.callback_click)
        t.dlink((self.model, 'sel_list'), (self.scatter, 'selected'))
        
    def callback_click(self, scatter, change):
        
        idx = change['data']['index']
        
        tmp = self.model.sel_list[:]
        tmp +=[idx]
        self.model.sel_list = tmp

In [14]:
view2 = ScatterView2()

In [15]:
# Selecting multiple with regular click
view2

A Jupyter Widget

In [17]:
view2.model.sel_list

[0, 1]

In [18]:
view2.scatter.selected

[0, 1]